0 导入包

In [3]:
import torch

# 加载数据的包
from datasets import load_from_disk

# 加载模型的包
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# 训练相关的包
from trl import SFTTrainer
from transformers import TrainingArguments
from peft import LoraConfig

1 加载数据

In [4]:
dataset = load_from_disk('3token_dataset')
dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'input_ids', 'attention_mask'],
        num_rows: 3408
    })
    test: Dataset({
        features: ['prompt', 'input_ids', 'attention_mask'],
        num_rows: 852
    })
})

In [5]:
# 选择其中的一部分数据
train_data = dataset['train'].shuffle(seed=42).select(range(2000)) # 随机选择一些数据
test_data = dataset['test'].shuffle(seed=42).select(range(800)) # 随机选择一些数据

2 量化加载模型

In [6]:
model_path = "google/gemma-2b-it" # 模型名字或路径
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,

    # 参数与显存
    device_map={"": 0}, # k代表参数名前缀，空代表所有模型参数 v代表在哪张gpu上运行

    # 量化加载配置
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True, # 加载到显存中的精度
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16 # 运算的精度
    ),
)

D:\0env_config\1py\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

3 训练配置

In [8]:
trainer = SFTTrainer(
    # 模型
    model=model,

    # 数据集
    train_dataset=train_data,
    eval_dataset=test_data,

    # 最大句子长度
    max_seq_length=2048,

    # 训练列名称
    dataset_text_field="prompt",

    packing=False,

    args=TrainingArguments(
        # ————————模型保存——————————
        output_dir="4lora",  # 模型保存路径
        overwrite_output_dir=True, # 同名是否覆盖
        save_strategy="steps", # 以步数为单位保存
        save_steps=25, # 多少步保存一次

        # ————————训练量————————————
        max_steps=300,  # 总训练次数
        per_device_train_batch_size=1, # 批次
        gradient_accumulation_steps=4,  # 累计多少次梯度更新
        group_by_length=True, # 将长度相似的放在一起训练

        # ————————训练精度———————————
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),

        # ————————更新参数———————————
        learning_rate=2e-4,  # 建议 2e-4 用于4位int量化，1e-4 用于8位int量化
        lr_scheduler_type="cosine",  # 学习率调度类型。也可以用线性： "linear"
        warmup_steps=5, # 学习率热身步数
        weight_decay=0.01,# 权重衰减系数

        optim="paged_adamw_8bit",  #优化器

        # ————————验证参数—————————————
        evaluation_strategy="steps", # 按照步数评估
        eval_steps=30,  # 每多少次循环评估一次
        prediction_loss_only=True, # 只返回预测损失
        eval_accumulation_steps=1, # 评估时的梯度累计

        # —————————log参数————————————
        logging_steps=25, # 多少步记录一次日志
        logging_dir="./logs", # 日志目录
        report_to=None, # 不把数据报给任何平台

        # ——————————种子————————————————
        seed=1337,

    ),

    peft_config=LoraConfig(
        r=8, # (rank)lora矩阵的大小，full rank等于全残微调
        lora_alpha=32,  # LoRA在原始基础上的学习率缩放
        target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
        task_type="CAUSAL_LM",  # 指定任务类型为因果
        lora_dropout=0.05,
        bias="none",
    ),
)

4 开始训练

In [9]:
trainer.train()

Step,Training Loss,Validation Loss


D:\0env_config\1py\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
D:\0env_config\1py\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
D:\0env_config\1py\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
D:\0env_config\1py\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0

TrainOutput(global_step=300, training_loss=1.135445540746053, metrics={'train_runtime': 192.128, 'train_samples_per_second': 6.246, 'train_steps_per_second': 1.561, 'total_flos': 1264241313361920.0, 'train_loss': 1.135445540746053, 'epoch': 0.6})

5 训练完了清空缓存

In [10]:
torch.cuda.empty_cache()